# Creating a Personalized Maplist Scraper for Splatoon3

In [38]:
import os 
import pandas as pd
import numpy as np
from lxml import html
from lxml import etree
import requests
from bs4 import BeautifulSoup
import re
import datetime

In [39]:
page_url = "https://splatoon3.ink/data/schedules.json"
start_time = []
end_time = []
mode_list = []
map_list = []
text = requests.get(page_url).text
text

import json
data = json.loads(text)

data["data"]["bankaraSchedules"]["nodes"][0] #entire dataset = multiple of these

{'startTime': '2023-10-19T20:00:00Z',
 'endTime': '2023-10-19T22:00:00Z',
 'bankaraMatchSettings': [{'__isVsSetting': 'BankaraMatchSetting',
   '__typename': 'BankaraMatchSetting',
   'vsStages': [{'vsStageId': 12,
     'name': 'Mahi-Mahi Resort',
     'image': {'url': 'https://splatoon3.ink/assets/splatnet/v2/stage_img/icon/low_resolution/8273118c1ffe1bf6fe031c7d8c9795dab52632c9b76e8e9f01f644ac5ae0ccc0_1.png'},
     'id': 'VnNTdGFnZS0xMg=='},
    {'vsStageId': 18,
     'name': 'Manta Maria',
     'image': {'url': 'https://splatoon3.ink/assets/splatnet/v2/stage_img/icon/low_resolution/0b7fd997781e03eb9d5bf1875ed070f698afc654f4fe929452c65aa26c0a35fd_1.png'},
     'id': 'VnNTdGFnZS0xOA=='}],
   'vsRule': {'name': 'Tower Control', 'rule': 'LOFT', 'id': 'VnNSdWxlLTI='},
   'bankaraMode': 'CHALLENGE'},
  {'__isVsSetting': 'BankaraMatchSetting',
   '__typename': 'BankaraMatchSetting',
   'vsStages': [{'vsStageId': 12,
     'name': 'Mahi-Mahi Resort',
     'image': {'url': 'https://splatoon3.

In [40]:
for entry in data["data"]["bankaraSchedules"]["nodes"]:
    start_time.append(entry["startTime"])
    #end_time.append(entry["endTime"])
    for subentry in entry["bankaraMatchSettings"]: #["bankaraMatchSettings"]:
        if subentry["bankaraMode"] == "OPEN":
            print(subentry["vsRule"]["name"])
            mode_list.append(subentry["vsRule"]["name"])
            for trientry in subentry["vsStages"]:
                map_list.append(trientry["name"])

Rainmaker
Splat Zones
Rainmaker
Tower Control
Splat Zones
Clam Blitz
Rainmaker
Tower Control
Splat Zones
Clam Blitz
Rainmaker
Tower Control


In [41]:
mode_list

['Rainmaker',
 'Splat Zones',
 'Rainmaker',
 'Tower Control',
 'Splat Zones',
 'Clam Blitz',
 'Rainmaker',
 'Tower Control',
 'Splat Zones',
 'Clam Blitz',
 'Rainmaker',
 'Tower Control']

In [42]:
map_1 = []
map_2 = []
for i in range(len(map_list)):
    if i%2==0:
        map_1.append(map_list[i])
    else:
        map_2.append(map_list[i])

In [43]:
#pairs = []
#for i in range(0, len(map_list), 2):
#    pairs.append((map_list[i], map_list[i+1]))

#pairs
    
#L = [1, 2, 3, 4, 5, 6]
#L2 = []

#for i in range(0, len(L), 2):
# L2.append((L[i], L[i+1]))

In [44]:
len(mode_list)

12

In [45]:
import pandas as pd
#full_data = {"title" : title_list, "rank_no" : rank_list,"price":price_list,'appid':appid_list,'bundleid':bundleid_list,'crtrids':crtrids_list}
full_data = {"startTime": start_time[:13], "mode":mode_list, "map1": map_1, "map2": map_2} #"endTime": end_time[:12]
full_df=pd.DataFrame(full_data)
full_df.to_csv("splatoon_maps.csv")


In [46]:
splatoon = pd.read_csv("splatoon_maps.csv")
splatoon = splatoon.drop(columns=["Unnamed: 0"])
splatoon

,startTime,mode,map1,map2
0,2023-10-19T20:00:00Z,Rainmaker,Mahi-Mahi Resort,Wahoo World
1,2023-10-19T22:00:00Z,Splat Zones,Hammerhead Bridge,MakoMart
2,2023-10-20T00:00:00Z,Rainmaker,Barnacle & Dime,Museum d'Alfonsino
3,2023-10-20T02:00:00Z,Tower Control,Mincemeat Metalworks,Flounder Heights
4,2023-10-20T04:00:00Z,Splat Zones,Undertow Spillway,Um'ami Ruins
5,2023-10-20T06:00:00Z,Clam Blitz,Hagglefish Market,Brinewater Springs
6,2023-10-20T08:00:00Z,Rainmaker,Eeltail Alley,Shipshape Cargo Co.
7,2023-10-20T10:00:00Z,Tower Control,Manta Maria,Crableg Capital
8,2023-10-20T12:00:00Z,Splat Zones,Undertow Spillway,Sturgeon Shipyard
9,2023-10-20T14:00:00Z,Clam Blitz,Mincemeat Metalworks,Humpback Pump Track


In [47]:
from datetime import datetime
import pytz

#datetime.fromisoformat('2020-01-06T00:00:00.000Z'[:-1] + '+00:00')
#datetime.datetime(2020, 1, 6, 0, 0, tzinfo=datetime.timezone.utc)

first = splatoon["startTime"].values[0]

values = []
for i in splatoon["startTime"].values:
    d = datetime.fromisoformat(i[:-1]) #.astimezone(timezone.utc)
    
    utc = pytz.utc.localize(d)
    pst = utc.astimezone(pytz.timezone("America/Los_Angeles"))
    values.append(pst.strftime("%H:%M"))

values

['13:00',
 '15:00',
 '17:00',
 '19:00',
 '21:00',
 '23:00',
 '01:00',
 '03:00',
 '05:00',
 '07:00',
 '09:00',
 '11:00']

In [48]:
splatoon["time"] = values
splatoon = splatoon[["time", "mode", "map1", "map2"]]

In [49]:
splatoon.to_csv("rotations.csv") #save

### All Map Modes:

In [50]:
splatoon

,time,mode,map1,map2
0,13:00,Rainmaker,Mahi-Mahi Resort,Wahoo World
1,15:00,Splat Zones,Hammerhead Bridge,MakoMart
2,17:00,Rainmaker,Barnacle & Dime,Museum d'Alfonsino
3,19:00,Tower Control,Mincemeat Metalworks,Flounder Heights
4,21:00,Splat Zones,Undertow Spillway,Um'ami Ruins
5,23:00,Clam Blitz,Hagglefish Market,Brinewater Springs
6,01:00,Rainmaker,Eeltail Alley,Shipshape Cargo Co.
7,03:00,Tower Control,Manta Maria,Crableg Capital
8,05:00,Splat Zones,Undertow Spillway,Sturgeon Shipyard
9,07:00,Clam Blitz,Mincemeat Metalworks,Humpback Pump Track


### Now, pick 1-2 maps you want to know the schedule of for each mode (i.e. map preferences)

In [51]:
zones = splatoon[splatoon["mode"]=="Splat Zones"]
maps_zones_1 = zones[(zones["map1"]=="Shipshape Cargo Co.") | (zones["map2"]=="Hagglefish Mrket")]
maps_zones_2 = zones[(zones["map2"]=="Shipshape Cargo Co.") | (zones["map1"]=="Hagglefish Mrket")]

tower = splatoon[splatoon["mode"]=="Tower Control"]
maps_tower_1 = tower[(tower["map1"]=="Sturgeon Shipyard") | (tower["map2"]=="Inkblot Art Academy")]
maps_tower_2 = tower[(tower["map2"]=="Sturgeon Shipyard") | (tower["map1"]=="Inkblot Art Academy")]

rain = splatoon[splatoon["mode"]=="Rainmaker"]
maps_rain_1 = rain[(rain["map1"]=="Eeltail Alley") | (rain["map2"]=="Humpback Pump Track")]
maps_rain_2 = rain[(rain["map2"]=="Eeltail Alley") | (rain["map1"]=="Humpback Pump Track")]

clams = splatoon[splatoon["mode"]=="Clam Blitz"]
maps_clams_1 = clams[(clams["map1"]=="Scorch Gorge") | (clams["map2"]=="Scorch Gorge")]
maps_clams_2 = None
#clams[(clams["map2"]=="Museum d'Alfonsino") | (clams["map1"]=="Scorch Gorge")]


all_modes = pd.concat([maps_zones_1, maps_zones_2, maps_tower_1, maps_tower_2, maps_rain_1, maps_rain_2, 
                       maps_clams_1, maps_clams_2], join="inner")

#all_modes = pd.concat([maps_zones_1, maps_zones_2, maps_tower_1, maps_tower_2, maps_rain_1, maps_rain_2, 
#                       maps_clams_1, maps_clams_2], join="inner")

## Result Table:

In [52]:
all_modes.sort_values(by=["time"]).reset_index().drop(columns=["index"])

,time,mode,map1,map2
0,01:00,Rainmaker,Eeltail Alley,Shipshape Cargo Co.


### **Finding:** There are usually 1-4 rotations that match one's map preferences for all modes.